In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
sys.path.append('../misc')
from MoviaBusDataset import MoviaBusDataset
import glob
files = sorted(glob.glob('../data/train/*/vehicle-position-matched-online.csv'))
all_data = None
for file in files:
    dataframe = MoviaBusDataset.parse_file(file)
    all_data = dataframe if all_data is None else pd.concat([all_data,dataframe]) 

In [2]:
data = pd.DataFrame(dataframe.groupby(['LinkRef'])['VehicleRef'].count())
data.rename(inplace=True,columns={'VehicleRef': 'freq' })

In [3]:
print('freq: min={}, max={}, mean={}, std={}'.format(data.min()['freq'],data.max()['freq'],data.mean()['freq'],data.std()['freq']))

freq: min=56, max=11938, mean=1887.7864583333333, std=1979.307018512525


In [4]:
#print(data.min()
import numpy as np
with np.load('../dcrnn.npz') as dcrnn_data:
    # also contains groundtruth
    out_dcrnn = dcrnn_data['predictions']
    data['t1'],data['t2'],data['t3'],data['t4'],data['t5'],data['t6'] = out_dcrnn.mean(axis=1)

In [5]:
data.index

Index(['103524893:266482296:1195317079', '103524902:1242123651:1195317083',
       '103524902:1242123670:3396253208', '103524902:3396253208:1242123651',
       '108233932:576722:4279694303', '108233935:4279694307:1195317078',
       '108233940:4459313602:4279694306', '115678954:1306239263:576726',
       '115678957:282678754:1306239519', '115678974:10939966:282678730',
       ...
       '87451616:4121323609:4929002366', '87451616:4929002366:4121323609',
       '87502481:4279694293:4279694295', '87502481:4279694295:4279694293',
       '87502482:1614362060:1614362080', '87502482:1614362080:1614362060',
       '87724803:20908158:20908159', '87724803:20908158:792844',
       '87724803:20908159:20908158', '87724803:792844:20908158'],
      dtype='object', name='LinkRef', length=192)

In [6]:
train = MoviaBusDataset('../data/train')

In [7]:
train.__getitem__(0)['data'].shape

torch.Size([1, 192])

In [8]:
import folium
import branca
import numpy as np
import pandas as pd
from MoviaBusDataset import MoviaBusDataset
import matplotlib.pyplot as plt
linkrefs = data.index
linkrefs = [linkref.split(':') for linkref in  linkrefs]
wayID = [linkrefs[i].pop(0) for i in range(len(linkrefs))]
#Create a map object in valby
#print(wayID)
m = folium.Map(location=[55.66,12.515], zoom_start=15)
def linkref_filter(feature):
    wayid = feature['properties']['WayId']
    #print(yolo)
    #print(wayid)
    if str(wayid) in wayID:
        return {
                'strokeOpacity': 0.01,
                'weight': 2,
                'color':'red'
        }
    else:
        val = 0
        return {
                'fillOpacity': 0,
                'weight': 0,
                'color':0
        }
    
        
#Add the geojson with colors
g=folium.GeoJson('../data/road_network.geojson',name='map', style_function=linkref_filter)
#g.data
folium.GeoJsonTooltip(['WayId','Source','Target','Oneway']).add_to(g)
g.add_to(m)
#bus_stop_layer = folium.GeoJson('../data/Bus/bus_stoppoint.geojson',name='bus stops', style_function=lambda x: {'fillOpacity': 0.1,
#                'weight': 5,
#                'color':'blue'})
#folium.GeoJsonTooltip(['Ref']).add_to(bus_stop_layer)
#bus_stop_layer.add_to(m)
#This is a hack to show a colorbar, and also include all the streets
#m.choropleth(geo_data='../data/road_network.geojson'
#             #, data=train.dataframes[0], columns=['LinkRef']
#             , key_on='feature.properties.WayId'
#             , fill_color='YlOrRd'
#             , name='Streets'
#             , legend_name='Mean speed [m/s]')
#Add a control layer to our map
folium.LayerControl().add_to(m)

#Show the resulting map
m

In [9]:
#traffic_lights['448962778:17710382:1085760670','448963774:1696119295:17710382','87398581:21788483:1282580772','87398581:1282580772:21788483','87398581:812557:1696110319','87398581:1696110319:812557','87398581:8121572:1411071138','87398581:1411071138:8121572','87398581:8121576:8121711','87398581:8121711:8121576','448963785:1065827673:4459313602','4489637837:2599647834:4459313608','448963789:1242123649:4459313610','500955362:73095632:73099256','']

NameError: name 'traffic_lights' is not defined

In [10]:
#['428851686:576717:576718','428851686:576718:576717']
files = sorted(glob.glob('../data/train/*/vehicle-position-matched-online.csv'))
bus_stops = None
for file in files:
    dataframe = MoviaBusDataset.parse_file(file,remove_stops=False)
    bus_stops = dataframe if all_data is None else pd.concat([all_data,dataframe]) 

In [11]:
#data['stops'] = bus_stops[['LinkRef','StopPointRef']].fillna(0).groupby(['LinkRef']).apply(lambda x: 1 if x else 0 )
data['stops'] = bus_stops[['LinkRef','StopPointRef']].fillna(0).groupby(['LinkRef']).apply(lambda x: int(bool(x['StopPointRef'].max())))

In [12]:
train = MoviaBusDataset('../data/train',agg_time=10)

In [13]:
min10_data =  MoviaBusDataset('../data/train', interpolation=True, 
                       prev_timesteps=6, 
                       max_future_time_steps=6,agg_time=10)

In [14]:
data['stds'] = pd.concat(min10_data.dataframes).std()

In [16]:
data['mean_diff'] = pd.concat(min10_data.dataframes).diff().mean()

In [15]:
data['mean_diff_diff'] = pd.concat(min10_data.dataframes).diff().diff().mean()

In [17]:
data['mean'] = pd.concat(min10_data.dataframes).mean()

In [18]:
import matplotlib.pyplot as plt
data_norm = (data.sub(data.mean(axis=0,numeric_only=True),axis=1))/data.std()
#data.head()
#plt.matshow()
#data_norm
data_norm.corr()

,freq,t1,t2,t3,t4,t5,t6,stops,stds,mean_diff_diff,mean_diff,mean
freq,1.000000,-0.086560,-0.119751,-0.134639,-0.139214,-0.139939,-0.140606,0.203034,-0.047307,-0.078915,-0.097765,-0.429810
t1,-0.086560,1.000000,0.987194,0.966518,0.947959,0.930464,0.913371,-0.163340,-0.269850,-0.240759,0.067034,0.148182
t2,-0.119751,0.987194,1.000000,0.993426,0.981275,0.967209,0.952776,-0.200761,-0.252915,-0.224726,0.077834,0.198384
t3,-0.134639,0.966518,0.993426,1.000000,0.996099,0.987216,0.976214,-0.229102,-0.231834,-0.204182,0.094682,0.217439
t4,-0.139214,0.947959,0.981275,0.996099,1.000000,0.997163,0.990173,-0.245932,-0.221765,-0.193706,0.106178,0.224312
t5,-0.139939,0.930464,0.967209,0.987216,0.997163,1.000000,0.997606,-0.254535,-0.221879,-0.189018,0.114903,0.229698
t6,-0.140606,0.913371,0.952776,0.976214,0.990173,0.997606,1.000000,-0.258901,-0.228306,-0.186995,0.121648,0.237094
stops,0.203034,-0.163340,-0.200761,-0.229102,-0.245932,-0.254535,-0.258901,1.000000,-0.038347,0.051875,-0.167941,-0.511415
stds,-0.047307,-0.269850,-0.252915,-0.231834,-0.221765,-0.221879,-0.228306,-0.038347,1.000000,0.114886,0.084958,-0.246023
mean_diff_diff,-0.078915,-0.240759,-0.224726,-0.204182,-0.193706,-0.189018,-0.186995,0.051875,0.114886,1.000000,0.140762,0.007169
